Модель  Logistic Regression 

In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
from sklearn.impute import SimpleImputer

# Завантаження даних
df = pd.read_csv(r'D:\IT\GoIT\Data Science\Project Data Science\Classification_of_Toxic_Comments\archive\internet_service_churn.csv')

# Попередня обробка даних (закодування категоріальних змінних)
df = pd.get_dummies(df, drop_first=True)

# Імпутація пропущених значень (замінимо пропущені значення на середнє для числових змінних)
imputer = SimpleImputer(strategy='mean')
X = imputer.fit_transform(df.drop('churn', axis=1))  # Все крім стовпця 'Churn'
y = df['churn']  # Цільова змінна

# Розділяємо дані на тренувальний і тестовий набори
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Нормалізація даних (StandardScaler)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Вибір алгоритму (Logistic Regression)
model = LogisticRegression(random_state=42, max_iter=1000)

# Крос-валідація з RandomizedSearch для пошуку найкращих параметрів
param_grid = {
    'C': np.logspace(-4, 4, 20),  # Гіперпараметр регуляризації
    'solver': ['lbfgs', 'liblinear'],  # Різні алгоритми оптимізації
    'penalty': ['l2'],  # Вид регуляризації
}

# Використовуємо RandomizedSearchCV для пошуку найкращих параметрів
random_search = RandomizedSearchCV(estimator=model, param_distributions=param_grid, 
                                   n_iter=10, cv=3, scoring='accuracy', n_jobs=-1, verbose=2, random_state=42)
random_search.fit(X_train, y_train)

# Найкращі параметри
best_model = random_search.best_estimator_
print(f'Найкращі параметри: {random_search.best_params_}')

# Прогнозування
y_pred = best_model.predict(X_test)

# Оцінювання моделі
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print(f'Accuracy: {accuracy:.4f}')
print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1 Score: {f1:.4f}')

# Виведення докладного звіту
print(classification_report(y_test, y_pred))

# Матриця неточностей
conf_matrix = confusion_matrix(y_test, y_pred)
print(f'Матриця неточностей:\n{conf_matrix}')


Fitting 3 folds for each of 10 candidates, totalling 30 fits
Найкращі параметри: {'solver': 'lbfgs', 'penalty': 'l2', 'C': np.float64(29.763514416313132)}
Accuracy: 0.8879
Precision: 0.9312
Recall: 0.8584
F1 Score: 0.8933
              precision    recall  f1-score   support

           0       0.84      0.92      0.88      6553
           1       0.93      0.86      0.89      7902

    accuracy                           0.89     14455
   macro avg       0.89      0.89      0.89     14455
weighted avg       0.89      0.89      0.89     14455

Матриця неточностей:
[[6052  501]
 [1119 6783]]
